# FIND USER'S NAME AND ACCOUNT LINKS BASED ON KEYWORD

#### Import necessary libraries

In [1]:
# for scraping 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from bs4 import BeautifulSoup as bs

# misc
import re as re # regex 
import time
import pandas as pd

# accesing env file 
import os 
from dotenv import load_dotenv # to access the secret keys we've hidden in a separate file 
load_dotenv() # grab values inside env file

True

#### Selenium and LinkedIn Setup

In [2]:
PATH = os.getenv("WEBDRIVER_PATH")
USERNAME = os.getenv("LI_USERNAME")
PASSWORD = os.getenv("LI_PASS")


In [3]:
# initialize web driver that would control the web browser
ser = Service(PATH)
op = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=ser, options=op)

# website we wanted to access 
driver.get("https://www.linkedin.com/uas/login")
time.sleep(3) # added a pause to avoid getting marked as bot 

# login in linkedin
email=driver.find_element(By.ID,"username")
email.send_keys(USERNAME)
password=driver.find_element(By.ID,"password")
password.send_keys(PASSWORD)
time.sleep(3)
password.send_keys(Keys.RETURN)

#### Web Scraping Process

In [17]:
user_links = []
user_names = []
user_location = []
user_bio = []

def user_find_scrape(search_info, max_page):
    time.sleep(5)
    
    # scroll mindlessly to avoid getting flagged as a bot
    start=time.time()
    lastHeight = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        newHeight = driver.execute_script("return document.body.scrollHeight")
        if newHeight == lastHeight:
            break
        lastHeight = newHeight
        end=time.time()
        if round(end-start)>20: # scrolling thru website - pretending we're not a bot for 20 seconds 
            break

    # type into search bar 
    search_bar = driver.find_element(By.CLASS_NAME, "search-global-typeahead__input")
    search_bar.send_keys(search_info)
    time.sleep(5)
    search_bar.send_keys(Keys.ENTER)
    time.sleep(5)

    # expand search result 
    try: 
        expand_result_click = driver.find_element(By.LINK_TEXT, "See all people results").click()
        time.sleep(5)
    except:
        print("Expand result failure")
        driver.quit()

    page = 0
    while page <= max_page: # how many search result page | each page has 10 users
        # find container for all names available in the current page  
        expanded_page_result = driver.page_source
        linkedin_soup = bs(expanded_page_result.encode("utf-8"), "html")
        linkedin_soup.prettify()
        huge_container = linkedin_soup.find("ul",{"class":"reusable-search__entity-result-list"})
        containers = huge_container.findAll("li",{"class":"reusable-search__result-container"})

        # iterate thru each of the name - grab each user's specified value and append each user's value in an array 
        iterations = 0
        nos = 5 # grab only 5 users on each page
        for user in containers: 
            try:
                user_name_box = user.find("span",{"class":"entity-result__title-text"})
                user_name_value = user_name_box.find("span",{"aria-hidden":"true"}).get_text()
                user_link_value = user_name_box.find("a",{"class":"app-aware-link"}, href=True)["href"]
                user_location_value = user.find("div",{"class":"entity-result__secondary-subtitle"}).get_text()
                user_bio_value = user.find("div",{"class":"entity-result__primary-subtitle"}).get_text()

                user_links.append(user_link_value)
                user_names.append(user_name_value)
                user_location.append(user_location_value)
                user_bio.append(user_bio_value)

                iterations += 1

                if(iterations==nos):
                    break

            except:
                print("There is an error fetching users. Initiating exit.")
                driver.quit()
        
        # click next button - FIX
        # try: 
        time.sleep(5)
        print(driver.page_source)
        next_page_click = driver.find_element_by_class_name("artdeco-pagination__button--next")
        next_page_click.click()
        page += 1
        time.sleep(5)
        # except:
        #     print("Next button clicked error. Initiating exit.")
        #     driver.quit()
        
    

            

#### Execute scraping

In [18]:
user_find_scrape("gdsc",5) # parameter is the keyword used for searching 
driver.quit()

# Using loop to execute scraping on multiple pages 
# n = int(len(account_links))
# for j in range(n):
#     user_find_scrape(account_links[j],"gdsc")

# Quit 
# driver.quit()

<html lang="en" class="theme theme--mercado artdeco windows"><head>
    <script type="text/javascript" async="" charset="utf-8" id="utag_128" src="https://platform.linkedin.com/litms/vendor/google/gtag-adwords.js?id=AW-867846157"></script><script type="application/javascript">!function(i,n){void 0!==i.addEventListener&&void 0!==i.hidden&&(n.liVisibilityChangeListener=function(){i.hidden&&(n.liHasWindowHidden=!0)},i.addEventListener("visibilitychange",n.liVisibilityChangeListener))}(document,window);</script>

    <title>"gdsc" | Search | LinkedIn</title>

    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="asset-url" class="mercado-icons-sprite" id="artdeco-icons/static/images/sprite-asset" content="https://static-exp1.licdn.com/sc/h/7438dbnn8galtczp2gk2s4bgb">
    <meta name="description" content="">
    <meta name="google" content="notranslate">
    <meta name="service" content="voyager-web">
    <meta name="theme-color" content="#ffff

C:\Users\Admin\AppData\Local\Temp/ipykernel_6516/1806604107.py:76: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  next_page_click = driver.find_element_by_class_name("artdeco-pagination__button--next")


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".artdeco-pagination__button--next"}
  (Session info: chrome=96.0.4664.45)
Stacktrace:
Backtrace:
	Ordinal0 [0x01156903+2517251]
	Ordinal0 [0x010EF8E1+2095329]
	Ordinal0 [0x00FF2848+1058888]
	Ordinal0 [0x0101D448+1233992]
	Ordinal0 [0x0101D63B+1234491]
	Ordinal0 [0x01047812+1406994]
	Ordinal0 [0x0103650A+1336586]
	Ordinal0 [0x01045BBF+1399743]
	Ordinal0 [0x0103639B+1336219]
	Ordinal0 [0x010127A7+1189799]
	Ordinal0 [0x01013609+1193481]
	GetHandleVerifier [0x012E5904+1577972]
	GetHandleVerifier [0x01390B97+2279047]
	GetHandleVerifier [0x011E6D09+534521]
	GetHandleVerifier [0x011E5DB9+530601]
	Ordinal0 [0x010F4FF9+2117625]
	Ordinal0 [0x010F98A8+2136232]
	Ordinal0 [0x010F99E2+2136546]
	Ordinal0 [0x01103541+2176321]
	BaseThreadInitThunk [0x7577FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77557A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77557A6E+238]


#### Storing scraped data into specified file format

In [ ]:
data = {
    "Name": user_names,
    "Location": user_location,
    "Bio": user_bio,
    "Link": user_links
}

# save to csv 
df = pd.DataFrame(data)
df.head(5)
df.to_csv("user_find.csv", encoding='utf-8', index=False)

# save to excel 
# writer = pd.ExcelWriter("user_posts.xlsx", engine='xlsxwriter')
# df.to_excel(writer, index =False)
# writer.save()